# Steps towards developing a postgres dol

We're going to need a database to play with, so first let's get that! 

We don't like to show secrets in our code, so we'll use [config2py](https://pypi.org/project/config2py/) to specify our DB keys.

In [5]:
import config2py

POSTGRESS_TEST_DB_URL = config2py.config_getter('POSTGRESS_TEST_DB_URL')

## SQLAlchemy

We'll use [SQLAlchemy](https://pypi.org/project/SQLAlchemy/) to interface with postgress from python.

Let's first look at what this sqlalchemy interface looks like with our test table, 
doing some basic operations that we'll won't to DOLify later.

### get list of table names

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine.reflection import Inspector

# Replace the placeholder values with your actual database credentials
engine = create_engine(POSTGRESS_TEST_DB_URL)

inspector = Inspector.from_engine(engine)

# List table names
table_names = inspector.get_table_names()
print(table_names)

/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ipykernel_65051/1019456399.py:7: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


['app', 'prompt_template', 'users']


## Get rows for a given table and selector

In a given table, get a generator the will yield rows given a selector (selection logic), by default the "everything" selector, yielding all rows

In [7]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker, Session

engine = create_engine(POSTGRESS_TEST_DB_URL)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Déclaration de la base pour la définition des modèles SQLAlchemy
Base = declarative_base()

class User(Base) :
    __tablename__ = "user"
    id = Column(Integer, primary_key=True)
    name = Column(String, index=True)
    token = Column(String, index=True)

class App(Base) :
    __tablename__ = "app"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)

class AppPermission(Base) :
    __tablename__ = "app_permission"
    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer)
    app_id = Column(Integer)

class PromptTemplate(Base) :
    __tablename__ = "prompt_template"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    template = Column(String)
    rjsf_ui = Column(String)

In [39]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select, text

Session = sessionmaker(bind=engine)
session = Session()

# Selection logic, by default fetching all
query = select(PromptTemplate).filter()  # Adjust the number per your needs

# for row in session.execute(query):
#     print(row)

cursor = session.execute(query)

print(f"{type(cursor)=}")

from typing import Iterator, Iterable, Generator
print(f"{isinstance(cursor, Generator)=}")
print(f"{isinstance(cursor, Iterator)=}")  # has __next__ method (so can do next(cursor))
print(f"{isinstance(cursor, Iterable)=}")  # has __iter__ method (so can do for x in cursor...)


type(cursor)=<class 'sqlalchemy.engine.result.ChunkedIteratorResult'>
isinstance(cursor, Generator)=False
isinstance(cursor, Iterator)=True
isinstance(cursor, Iterable)=True


In [12]:
from oa import prompt_function

prompt_function("""Suggest {n:30} names 
between {min_length:1} and {max_length:15} characters long for {thing}.
Only output the names, one per line with no words before or after it, 
since I will be parsing the output.""")

<function oa.tools.prompt_function.<locals>.ask_oa(thing, *, n='30', min_length='1', max_length='15')>

In [42]:
from sqlalchemy import create_engine, Table, MetaData, select

metadata = MetaData()

# Reflect the table from the database
prompt_template_table = Table('prompt_template', metadata, autoload_with=engine)

# Now you can use this table object to construct a query
query = select(prompt_template_table)

# Execute the query
with engine.connect() as connection:
    result = connection.execute(query)
    for row in result:
        print(row)  # Each row is a RowProxy object that allows for column access by name


(1, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}')
(2, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}')
(3, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}')
(4, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}')
(5, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}')
(6, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}')


## My first (read only) DOL

In [1]:
import config2py

POSTGRESS_TEST_DB_URL = config2py.config_getter('POSTGRESS_TEST_DB_URL')

URI = POSTGRESS_TEST_DB_URL

# An old class:
# from sqldol import SQLAlchemyPersister
# t = SQLAlchemyPersister(POSTGRESS_TEST_DB_URL, 'devdb')

def print_attr_doc(obj):
    for k in filter(lambda x: not x.startswith('_'), dir(obj)):
        v = getattr(obj, k)
        if doc := getattr(v, '__doc__'):
            # print the first 88 characters or until the first newline
            print(f" * {k}: {doc[:88].splitlines()[0]}")



In [3]:
example_data = {
    "name": ["Alice", "Bob", "Charlie", "Diana"],
    "age": [30, 25, 35, 28],
    "extras": [
        {"hobby": "cycling", "pet": "cat"},
        {"hobby": "hiking", "pet": "dog"},
        {"hobby": "swimming", "pet": "fish"},
        {"hobby": "reading"}
    ]
}

# Create the table with the example data and type mapping
# from sqldol.scrap.postgres_dol import DFLT_URI
from sqldol.util import create_table_from_dict
table = create_table_from_dict(example_data, table_name="sqldol_test_table", uri=URI)


In [2]:
from sqldol.base import *

tables = TablesDol(URI)
list(tables)

['app',
 'prompt_template',
 'users',
 'py2store_default_table',
 'devdb',
 'test_table']

In [5]:
table_obj = tables['test_table']

In [7]:
columns = TableColumnsDol(table_obj)
list(columns)

['name', 'age', 'extras']

In [14]:
column_obj = columns['age']
column_obj

Column('age', INTEGER(), table=<test_table>)

In [18]:
type(column_obj)

sqlalchemy.sql.schema.Column

In [17]:
# get list of column values
print_attr_doc(column_obj)

 * all_: Produce an :func:`_expression.all_` clause against the
 * allows_lambda: bool(x) -> bool
 * anon_key_label: A unicode subclass used to identify anonymously
 * anon_label: A unicode subclass used to identify anonymously
 * any_: Produce an :func:`_expression.any_` clause against the
 * argument_for: Add a new kind of dialect-specific keyword argument for this class.
 * asc: Produce a :func:`_expression.asc` clause against the
 * autoincrement: bool(x) -> bool
 * base_columns: frozenset() -> empty frozenset object
 * between: Produce a :func:`_expression.between` clause against
 * bitwise_and: Produce a bitwise AND operation, typically via the ``&``
 * bitwise_lshift: Produce a bitwise LSHIFT operation, typically via the ``<<``
 * bitwise_not: Produce a bitwise NOT operation, typically via the ``~``
 * bitwise_or: Produce a bitwise OR operation, typically via the ``|``
 * bitwise_rshift: Produce a bitwise RSHIFT operation, typically via the ``>>``
 * bitwise_xor: Produce a bitwi

/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ipykernel_94142/2234909852.py:13: SADeprecationWarning: The ColumnElement.anon_key_label attribute is now private, and the public accessor is deprecated. (deprecated since: 1.4)
  v = getattr(obj, k)
/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ipykernel_94142/2234909852.py:13: SADeprecationWarning: The ColumnElement.anon_label attribute is now private, and the public accessor is deprecated. (deprecated since: 1.4)
  v = getattr(obj, k)


In [21]:
table = TableColumnsCollection(table_obj)
list(table)

[Column('name', TEXT(), table=<test_table>),
 Column('age', INTEGER(), table=<test_table>),
 Column('extras', JSON(astext_type=Text()), table=<test_table>)]

In [34]:
column_obj = table[1]


KeyError: Column('age', INTEGER(), table=<test_table>)

In [6]:
column_obj = table[1]

 * all_: Produce an :func:`_expression.all_` clause against the
 * allows_lambda: bool(x) -> bool
 * anon_key_label: A unicode subclass used to identify anonymously
 * anon_label: A unicode subclass used to identify anonymously
 * any_: Produce an :func:`_expression.any_` clause against the
 * argument_for: Add a new kind of dialect-specific keyword argument for this class.
 * asc: Produce a :func:`_expression.asc` clause against the
 * autoincrement: bool(x) -> bool
 * base_columns: frozenset() -> empty frozenset object
 * between: Produce a :func:`_expression.between` clause against
 * bitwise_and: Produce a bitwise AND operation, typically via the ``&``
 * bitwise_lshift: Produce a bitwise LSHIFT operation, typically via the ``<<``
 * bitwise_not: Produce a bitwise NOT operation, typically via the ``~``
 * bitwise_or: Produce a bitwise OR operation, typically via the ``|``
 * bitwise_rshift: Produce a bitwise RSHIFT operation, typically via the ``>>``
 * bitwise_xor: Produce a bitwi

/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ipykernel_81893/511497009.py:11: SADeprecationWarning: The ColumnElement.anon_key_label attribute is now private, and the public accessor is deprecated. (deprecated since: 1.4)
  v = getattr(obj, k)
/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ipykernel_81893/511497009.py:11: SADeprecationWarning: The ColumnElement.anon_label attribute is now private, and the public accessor is deprecated. (deprecated since: 1.4)
  v = getattr(obj, k)


In [ ]:
print_attr_doc(table_obj)

In [ ]:
print_attr_doc(column_obj)

AttributeError: 'Table' object has no attribute 'bind'

In [201]:
type(table_obj), type(engine)

(sqlalchemy.sql.schema.Table, sqlalchemy.engine.base.Engine)

In [200]:
def table_iter(table, engine):
    query = select(table)
    with engine.connect() as connection:
        result = connection.execute(query)
        for row in result:
            yield row


t = list(table_iter(table_obj, engine))
t

[(1, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (2, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (3, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}'),
 (4, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (5, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (6, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}')]

In [171]:
t = s['prompt_template']
dir(t)
t.key, t.name

('prompt_template', 'prompt_template')

In [ ]:
engine = create_engine(POSTGRESS_TEST_DB_URL)

t = PostgressTables(POSTGRESS_TEST_DB_URL)

In [129]:
t = PostgresBaseColumnsReader(engine, 'prompt_template')
list(t)

['id', 'name', 'template', 'rjsf_ui']

In [133]:
t = PostgresBaseKvReader(engine, 'prompt_template', key_column='name', value_column='template')
list(t)

TypeError: tuple indices must be integers or slices, not str

In [79]:
t = PostgresTableRows(engine, 'prompt_template')    
list(t)


[(1, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (2, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (3, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}'),
 (4, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (5, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (6, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}')]

In [ ]:
user, prompt_template_name, prompt_template_string, rjsf_ui_spec

key-value views
--> user_prompt_templates
--> user_rjsf_ui_spec

In [ ]:
# def PostgresBaseKvReader(Mapping):
#     """A mapping view of a table, 
#     where keys are values from a key column and values are values from a value column.
#     There's also a filter function that can be used to filter the rows.
#     """
#     def __init__(
#             self, engine, table_name, 
#             key_column=None, 
#             value_column=None, 
#             filt=None

In [96]:
t = PostgresBaseColumnsReader(engine, 'prompt_template')
list(t)


['id', 'name', 'template', 'rjsf_ui']

In [ ]:
user_prompt_templates = MkUserPromptTemplateStore(user, ...)

list(user_prompt_templates)
user_prompt_templates['prompt_template_id']
user_prompt_templates['prompt_template_id'] = 'my prompt template'



In [110]:
# d = {'a': 1, 'b': 2}

# d.__getitem__('a')


# mk_app({user_store, methods=['__getitem__', '__setitem__']})

# ./user_store/prompt_templates/__getitem__?key=a

1

In [127]:
user, name, prompt_template_str, rjsf_ui_spec



NameError: name 'user' is not defined

In [59]:
tables = PostgressTables(engine)
list(tables)

['app', 'prompt_template', 'users']

In [51]:
t = tables['prompt_template']


In [54]:
list(t)

[(1, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (2, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (3, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}'),
 (4, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (5, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (6, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}')]

In [48]:
dir(t)

['__annotations__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__visit_name__',
 '__weakref__',
 '_all_selected_columns',
 '_annotate',
 '_annotations',
 '_annotations_cache_key',
 '_anonymous_fromclause',
 '_assert_no_memoizations',
 '_autoincrement_column',
 '_autoload',
 '_cache_key_traversal',
 '_clone',
 '_cloned_set',
 '_cols_populated',
 '_columns',
 '_compile_w_cache',
 '_compiler',
 '_compiler_dispatch',
 '_constructor',
 '_copy_internals',
 '_de_clone',
 '_deannotate',
 '_dialect_kwargs_traverse_internals',
 '_execute_on_connection',
 '_execute_on_scal